In [14]:
import numpy as np
import heapq
Distance = [[0,1064,673,1401,277],
            [1064,0,958,1934,377],
            [673,957,0,1001,399],
            [1401,1934,1001,0,387],
            [277,377,399,387,0]]
W_ = {1:{1:'A',2:'E',3:'D',4:'C',5:'A'},
        2:{1:'B',2:'E',3:'A',4:'C',5:'D'},
        3:{1:'B',2:'A',3:'B',4:'C',5:'E'},
        4:{1:'D',2:'A',3:'D',4:'B',5:'D'},
        5:{1:'B',2:'E',3:'C',4:'B',5:'D'}}
Starting_Point = ['A','B','C','D','E']
fac_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}
mp ={}

In [15]:
def extract_distance():
    for i in Starting_Point:
        dd = 20000
        for j in Starting_Point:
            if get_gscore_dist(i,j)<dd and i != j:
                dd = get_gscore_dist(i,j)
                mp[i]=j
#     print(mp)
    for i in Starting_Point:
        for j in Starting_Point:
            if get_gscore_dist(i,j)>(get_gscore_dist(i,mp[i])+get_gscore_dist(mp[i],j)):
                Distance[fac_dict[i]][fac_dict[j]]=get_gscore_dist(i,mp[i])+get_gscore_dist(mp[i],j)
#     print(Distance)
def backup_distance():
    Distance = [[0,1064,673,1401,277],
                [1064,0,958,1934,377],
                [673,957,0,1001,399],
                [1401,1934,1001,0,387],
                [277,377,399,387,0]]

In [16]:
class node():
    def __init__(self, cur_factory,last_table, gscore=float('inf'),fscore=float('inf'), finished_factory=set(), parent=''):
        self.gscore = gscore
        self.last_table = last_table
        self.cur_factory = cur_factory
        self.parent = parent
        self.fscore = fscore
        self.finished_factory = finished_factory
    
    def __key(self):
        return (self.parent)
    
    def __eq__(self, other):
        return (np.array_equal(self.last_table,other.last_table) and self.gscore <= other.gscore)
    
    def __lt__(self, other):
        return self.fscore < other.fscore
    
    def __hash__(self):
        return hash(self.__key())

In [17]:
def h_min_dis(node,factory):
    table = node.last_table.copy()
    temp = node.cur_factory
    table2 = table.copy()
    table2 = table2.T
    fscore = 0
    for num, i in enumerate (table):
        if table2[num].sum() != 0:
            fscore=fscore+Distance[fac_dict[temp]][fac_dict[mp[temp]]]
            temp = mp[temp]
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue   
    return table,fscore
#     table = node.last_table.copy()
#     for num, i in enumerate (table):
#         if i[0] == 1:
#             if W_[num+1][1] == factory: i[0] = i[0] - 1
#             continue
#         elif i[1] == 1:
#             if W_[num+1][2] == factory: i[1] = i[1] - 1 
#             continue
#         elif i[2] == 1:
#             if W_[num+1][3] == factory: i[2] = i[2] - 1 
#             continue
#         elif i[3] == 1:
#             if W_[num+1][4] == factory: i[3] = i[3] - 1
#             continue    
#         elif i[4] == 1:
#             if W_[num+1][5] == factory: i[4] = i[4] - 1 
#             continue
#     fscore = Distance[fac_dict[node.cur_factory]][fac_dict[factory]]
#     return table,fscore                                      
    
def h_min_step(node,factory):
    '''
    calculate the huristic of a certain step of node
    '''
    table = node.last_table.copy()
    table2 = table.copy()
    table2 = table2.T
    fscore = 0
    for num, i in enumerate (table):
        if table2[num].sum() != 0:
            fscore=fscore+1
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue
    return table,fscore

def get_gscore_dist(city1,city2):
    return Distance[fac_dict[city1]][fac_dict[city2]]

def get_close(table):
    s = ''
    for i in table:
        j = i.sum()
        s = s+ str(int(j))
    return s

def get_step(a,b):
    return 1

def get_low_rank(open_set):
    min_huris = float('inf')
    for nodes in open_set:
        if (nodes.fscore)<min_huris:
            cur_node = nodes
            min_huris = cur_node.fscore  
    return cur_node

def goal_test(node):
    if node.last_table.sum() == 0: return True
    return False

In [18]:
def A_star(start,hu,get_func):
    node_expanded=1
    hu_table = np.ones((5,5))
    initnode = node(start,hu_table,gscore=0,fscore=25.0)
    table,fscore = hu(initnode,start)
    start_node = node(start,table,gscore=0,fscore=fscore,parent=start)
    close_set = {}
    open_set = []
    heapq.heappush(open_set,start_node)
    cities = ['A','B','C','D','E']
    while len(open_set)!=0:
        cur_node = heapq.heappop(open_set) 
        if goal_test(cur_node): 
            print('node expanded=',node_expanded)
            return cur_node.parent
        close_set[get_close(cur_node.last_table)]=cur_node.gscore
        for city in cities:
            if city == cur_node.cur_factory: continue
            table,fscore = hu(cur_node,city)
            cost = cur_node.gscore+get_func(cur_node.cur_factory,city)
            if get_close(table) in close_set:
                if close_set[get_close(table)] < cost:
                    continue
                close_set[get_close(table)] = cost
            new_node = node(city,table,gscore=cost,fscore=fscore+cost,parent=cur_node.parent+city)
            heapq.heappush(open_set,new_node)
            node_expanded = node_expanded + 1
#             print(close_set)
    return

In [19]:
backup_distance()
for t in Starting_Point:
    lenn = 1000
    print('for starting =',t)
    retv = A_star(t,h_min_step,get_step)
    dis = 0
#     for s in retv:
#         dis = dis + get_gscore_dist(t,s)
#         t = s
#     print(dis)
    print('temp=',retv)
    print('len=',len(retv))
    print()

for starting = A
node expanded= 2704
temp= ABEDCADBCDE
len= 11

for starting = B
node expanded= 3925
temp= BDAEDCBACDE
len= 11

for starting = C
node expanded= 5314
temp= CBDAEDCBACED
len= 12

for starting = D
node expanded= 2018
temp= DABEDCABCDE
len= 11

for starting = E
node expanded= 5314
temp= EBDAEDCBACED
len= 12



In [39]:
extract_distance()
for t in Starting_Point:
    lenn = 1000
    print('for starting =',t)
    retv = A_star(t,h_min_dis,get_gscore_dist)
    startttt = t
    dis = 0
    new_sequence = ''
    for s in retv:
        if get_gscore_dist(startttt,s) != Distance2[fac_dict[startttt]][fac_dict[s]]:
            new_sequence = new_sequence + mp[s]
            new_sequence = new_sequence + s
        else:
            new_sequence = new_sequence + s
        dis = dis + get_gscore_dist(startttt,s)
        startttt = s
    print(retv)
    print('distance =', dis)
    print(new_sequence)
    print()

for starting = A
node expanded= 3825
ABEDCADBCED
distance = 5867
AEBEDECAEDEBECED

for starting = B
node expanded= 2037
BEDAEDBCEBAD
distance = 5726
BEDEAEDEBECEBEAED

for starting = C
node expanded= 3712
CBEDAEBDCEBAD
distance = 6502
CEBEDEAEBEDECEBEAED

for starting = D
node expanded= 1415
DBEAEBDCEBAD
distance = 5716
DEBEAEBEDECEBEAED

for starting = E
node expanded= 3741
EBEDAEBDCBAED
distance = 6103
EBEDEAEBEDECEBEAED



In [23]:
Distance

[[0, 654, 673, 664, 277],
 [654, 0, 776, 764, 377],
 [673, 776, 0, 786, 399],
 [664, 764, 786, 0, 387],
 [277, 377, 399, 387, 0]]

In [25]:
Distance2

[[0, 1064, 673, 1401, 277],
 [1064, 0, 958, 1934, 377],
 [673, 957, 0, 1001, 399],
 [1401, 1934, 1001, 0, 387],
 [277, 377, 399, 387, 0]]

In [32]:
dd = []
for a in range(5):
    cc = []
    for b in range(5):
        if Distance2[a][b] != 0:
            minnn = Distance2[a][b]
            for c in range(5):
                if Distance2[a][c] != 0:
                    if Distance2[a][c]+Distance2[c][b] < minnn:
                        minnn = Distance2[a][c]+Distance2[c][b]
            cc.append(minnn)
        else:
            cc.append(0)
    dd.append(cc)

In [45]:
backup_distance()
kk = 'DEBEAEBEDECEBEAED'
startttt = kk[0]
dis = 0
for s in kk:
        dis = dis + get_gscore_dist(startttt,s)
        startttt = s
print(dis)

5716


In [34]:
Distance

[[0, 654, 673, 664, 277],
 [654, 0, 776, 764, 377],
 [673, 776, 0, 786, 399],
 [664, 764, 786, 0, 387],
 [277, 377, 399, 387, 0]]

In [46]:
np.ones((5,5))

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])